In [20]:
type ∨[P, Q] = Either[P, Q]
type ∧[P, Q] = Tuple2[P, Q]
type ⟶[P, Q] = P => Q
type ⊥ = Nothing
type True = Unit
type ¬[P] = P => ⊥ 
type ⟷[P, Q] = (P => Q) ∧ (Q => P)

defined type ∨
defined type ∧
defined type ⟶
defined type ⊥
defined type True
defined type ¬
defined type ⟷

In [21]:
type Or[P, Q] = Either[P, Q]
type And[P, Q] = Tuple2[P, Q]
type Implies[P, Q] = P => Q
type False = Nothing
type True = Unit
type Not[P] = P => ⊥
type <=>[P, Q] = (P => Q, Q => P)

defined type Or
defined type And
defined type Implies
defined type False
defined type True
defined type Not
defined type <=>

In [29]:
class Inhabitant{ x => 
    // Knight(x) -- `x` is a Knight
    // 
    type Knight
    
    // Knave(x) -- `x` is a Knave (i.e. is not a Knight)
    // 
    type Knave = ¬[Knight]
    
    // Says(x, P) -- `x` says that `P` holds, i.e. asserts proposition `P`
    // 
    type Says[P]
    
    // Tipo Arthur York
    //
    type Arthur
    
    // Being always truthful
    type AlwaysTruthful = Knight    
}

defined class Inhabitant

In [30]:
trait KnightsKnaves{
    // P1. Inhabitants are knights or knaves
    // 
    //     ∀ x. Inhabitant(x) ⟶ Knight(x) ∨ Knave(x) 
    // 
    def P1(x: Inhabitant): x.Knight ∨ x.Knave
    
    // In Scala 3
    // val P1: (x: Inhabitant) => Either[x.Knight, x.Knave]
    
    // P2. Knights are truth tellers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ P
    // 
    def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
    
    // In Scala 3
    // val P2: [P] => (x: Inhabitant) => x.Knight => x.Says[P] => P
    
    // P3. Knaves are persistent liers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ ¬P
    // 
    def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
    
    /*P4. Si x dice que es Arthur York, sera un caballero porque dice la verdad - ¿?¿?
    def P4[P](x: Inhabitant): x.Arthur => x.Knight
    
    Tip de Juanma
    Si el sujeto es Arthur, entonces si Arthur dice P, quiere decir que el sujeto dice P
    def P4_1[P](x: Inhabitant)(arthur: Inhabitant): x.Arthur => arthur.Says[P] => x.Says[P]
    
    Si el sujeto es Arthur, entonces si Arthur es un Knight, quiere decir que el sujeto es un Knight
    def P4_2[P](x: Inhabitant)(arthur: Inhabitant): x.Arthur => arthur.Knight => x.Knight
    
    */
    
    //P4. If some inhabitant is actually Arthur York, all that can be proven about Arthur York can be proven about her
    val arthur: Inhabitant
    
    def P4_1[P](x: Inhabitant): x.Arthur => arthur.Says[P] => x.Says[P]
    def P4_2(x: Inhabitant): x.Arthur => arthur.Knight => x.Knight
    
    // P5. Double negation applies to things said
    def P5[P](x: Inhabitant): Not[Not[x.Says[P]]] => x.Says[P]
    
    // P6. Some inhabitant is or is not Arthur York
    def P6(x: Inhabitant): x.Arthur Or Not[x.Arthur]  
    
    // P7. Si x dice la verdad, x es Knight
    def P7[P](x: Inhabitant): x.Says[P] => P => x.Knight
    
    // P8. Si x miente, x es Knave
    def P8[P](x: Inhabitant): x.Says[P] => Not[P] => x.Knave
    
}

defined trait KnightsKnaves

In [39]:
implicit class SomeSugar(P: KnightsKnaves){    
    def eitherKnightOrKnave(x: Inhabitant): x.Knight Or x.Knave = 
        P.P1(x)
    
    def knaveIfNotKnight(x: Inhabitant): Not[x.Knight] => x.Knave = 
        notXIsKnight => 
            eitherKnightOrKnave(x).fold(notXIsKnight, identity)
    
    def knightIfNotKnave(x: Inhabitant): Not[x.Knave] => x.Knight = 
        notXIsKnave => 
            eitherKnightOrKnave(x).fold(identity, notXIsKnave)
    
    def knightsAreTruthTellers[P](x: Inhabitant): x.Knight => x.Says[P] => P = 
        P.P2[P](x)
    
    def knavesAreLiers[P](x: Inhabitant): x.Knave => x.Says[P] => Not[P] = 
        P.P3[P](x)  
    
    def doubleNotSayP[P](x: Inhabitant): Not[Not[x.Says[P]]] => x.Says[P] =
        P.P5[P](x)
        
    def eitherArthurOrNot(x: Inhabitant): x.Arthur Or Not[x.Arthur] = 
        P.P6(x)
  
   /*def knightIfTruthful[P](x: Inhabitant): x.Says[P] => P => x.Knight = 
        P.P7[P](x)
    */
    
    def knightIfTruthful[P](x: Inhabitant): x.Says[P] => P => x.Knight = 
        xSaysP => p => 
            eitherKnightOrKnave(x).fold(
                (xIsKnight: x.Knight) => xIsKnight, 
                (xIsKnave: x.Knave) => knavesAreLiers(x)(xIsKnave)(xSaysP)(p)
            )
    
    /*def knaveIfLier[P](x: Inhabitant): x.Says[P] => Not[P] => x.Knave = 
        P.P8[P](x)
    */
    def knaveIfLier[P](x: Inhabitant): x.Says[P] => Not[P] => x.Knave = 
        xSaysP => notP => eitherKnightOrKnave(x).fold(
            xIsKnight => notP(knightsAreTruthTellers(x)(xIsKnight)(xSaysP): P),
            identity)
}

implicit class NotSugar[P](notP: P => Nothing){
    def contradicts(p: P): Nothing = 
        notP(p)
}

defined class SomeSugar
defined class NotSugar

In [32]:
// { P1, P2, P3 } ⊢ ∀ x. Inhabitant(x) ⟶ ¬Says(x, Knave(x))

def puzzle1(premises: KnightsKnaves)(x: Inhabitant): x.Says[x.Knave] => Nothing =
    // 1. Says(x, Knave(x))                                       ; hypothesis
    ((_1: x.Says[x.Knave]) => 
         (premises.P1(x) match {
             // 2. Knight(x)                                      ; hypothesis
             case Left(_2: x.Knight) => 
                 //   3. Knight(x) → Says(x, Knave(x)) → Knave(x) ; P2[Knave(x),x]
                 val _3: x.Knight => x.Says[x.Knave] => x.Knave = premises.P2[x.Knave](x)
                 //   4. Says(x, Knave(x)) → Knave(x)             ; ⟶E(3,2)
                 val _4: x.Says[x.Knave] => x.Knave = _3(_2)
                 //   5. Knave(x)                                 ; ⟶E(4,1)
                 //   5. ¬ Knight(x)                              ; ≝ Knave
                 //   5. Knight(x) → ⊥                            ; ≝ ¬ 
                 val _5: x.Knight => ⊥ = _4(_1)
                 //   6. ⊥                                        ; ⟶E(5,2)
                 _5(_2) : ⊥

             //  _7. Knave(x)                                     ; hypothesis
             case Right(_7: x.Knave) => 
                 //   8. Knave(x) → Says(x, Knave(x)) → ¬ Knave(x) ; P3[Knave(x),x]
                 val _8: x.Knave => x.Says[x.Knave] => ¬[x.Knave] = premises.P3[x.Knave](x)
                 //   9. Says(x, Knave(x)) → ¬ Knave(x)            ; ⟶E(8,7)
                 val _9: x.Says[x.Knave] => ¬[x.Knave] = _8(_7)
                 //   10. ¬ Knave(x)                               ; ⟶E(9,1)
                 //   10. Knave(x) → ⊥                             ; ≝ ¬ 
                 val _10: x.Knave => ⊥ = _9(_1)
                 //  11. ⊥                                         ; ⟶E(10,7)
                 _10(_7) : ⊥

        // _12. ⊥                                                  ; ∨E(P1, 2-6, 7-11)
        }) : False

    // _13. Says(x, Knave(x)) ⟶ ⊥                                  ; ⟶I(1-12)
    // _13. ¬ Says(x, Knave(x))                                    ; ≝ ¬ 
    ) : ¬[x.Says[x.Knave]]

defined function puzzle1

# Problem 1 - The First Trial.
Inspector Craig of Scotland Yard—of whom you will read much in this book—was called to the Island of Knights and Knaves to help find a criminal named Arthur York. What made the process difficult was that it was not known whether Arthur York was a knight or a knave. 
One suspect was arrested and brought to trial. Inspector Craig was the presiding judge. Here is a transcript of the trial: 

Craig: What do you know about Arthur York? 
DEFENDANT: Arthur York once claimed that I was a knave. 

Craig: Are you by any chance Arthur York?
DEFENDANT: Yes.

Is the defendant Arthur York?

**SOLUCION**

If the defendant is Arthur York, we get the following contradiction. Suppose he is Arthur York. 
Then he is a knight, since he claimed to be Arthur York. That would mean that his first answer to Craig was also true, 
which means that he, Arthur York, once claimed that he was a knave. 

But that is impossible! **Therefore the defendant is not Arthur York, although he is, of course, a knave.**

In [41]:
/* El defendant en este caso será "x" y "Arthur York" será "arthur" tendremos:
Craig: What do you know about Arthur York? DEFENDANT: Arthur York once claimed that I was a knave. 
x.Says[premises.arthur.Says[x.Knave]] -> El defendant dice que Arthur dice que el defendant es un Knave

Craig: Are you by any chance Arthur York? DEFENDANT: Yes. -> 
x.Says[x.Arthur] -> El defendant dice que es Arthur

Solucion:

Si "x" es Arthur York, hay contradiccion. Si "x" es un knight porque se supone que ha dicho la verdad y es Arthur("a"), 
significa que ha dicho que es un knave. Eso es imposible. Por tanto "x" NO es Arthur York, y por tanto es un Knave.

La premisa seria: ((x.Says[premises.arthur.Says[x.Knave]], x.Says[x.Arthur])) => x.Knave
Y si "x" NO es Arthur York, entonces: x.Arthur => Nothing (o lo que es lo mismo Not[x.Arthur])

Finalmente la premisa seria: ((x.Says[premises.arthur.Says[x.Knave]], x.Says[x.Arthur])) => (x.Knave, Not[x.Arthur])*/

In [42]:
def problem1(premises: KnightsKnaves)(x: Inhabitant): 
    ((x.Says[premises.arthur.Says[x.Knave]], x.Says[x.Arthur])) => (x.Knave, Not[x.Arthur]) = ???

defined function problem1

In [47]:
//FINALIZADO OK 
//Simplificado v3

def problem1(premises: KnightsKnaves)(x: Inhabitant): 
    ((x.Says[premises.arthur.Says[x.Knave]], x.Says[x.Arthur])) => (x.Knave, Not[x.Arthur]) = 
    ({xSay => 
        val xIsNotArthur: Not[x.Arthur] = {
            xArthur =>
                val xIsKnight: x.Knight = 
                    premises.knightIfTruthful[x.Arthur](x)(xSay._2)(xArthur)                            
                val xSayTrue: premises.arthur.Says[x.Knave] = 
                    premises.knightsAreTruthTellers[premises.arthur.Says[x.Knave]](x)(xIsKnight)(xSay._1)                 
                val xSaySameArthur: x.Says[x.Knave] = 
                    premises.P4_1[x.Knave](x)(xArthur)(xSayTrue)
                
                puzzle1(premises: KnightsKnaves)(x)(xSaySameArthur)
        }
        val xIsKnave: x.Knave = premises.knaveIfLier[x.Arthur](x)(xSay._2)(xIsNotArthur)
        (xIsKnave, xIsNotArthur)
    })

defined function problem1